<h1>Deep Learning For NLP: Zero To Transformers & BERT</h1>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
from keras import utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [3]:
strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [8]:
source_folder = '/Users/shirabroner/Documents/Data Science /kaggle-data/jigsaw-multilingual-toxic-comment-classification'
train_df = pd.read_csv(source_folder + '/jigsaw-toxic-comment-train.csv')
validation_df = pd.read_csv(source_folder + '/validation.csv')
test_df = pd.read_csv(source_folder + '/test.csv')

In [14]:
train_df.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)
train_df

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1


In [15]:
train = train_df.loc[:12000,:]
train.shape

(12001, 3)

In [16]:
train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [17]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [18]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.comment_text.values, train.toxic.values, 
                                                  stratify=train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)